In [4]:
import grpc 

import tensorflow as tf 

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [5]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel=channel)

In [6]:
from keras_image_helper import create_preprocessor

In [10]:
preprocessor = create_preprocessor('xception',target_size=(299,299))

In [14]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [15]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [18]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [20]:
response = stub.Predict(pb_request,timeout=20.0)

In [25]:
preds = response.outputs['dense_7'].float_val

In [26]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [34]:
pred_dict = dict(zip(classes,preds))

pred_dict = {k:v for k,v in sorted(pred_dict.items(),key= lambda item: item[1],reverse=True)}

pred_dict

{'pants': 9.903780937194824,
 'shorts': 3.24115252494812,
 'outwear': -1.0892630815505981,
 'dress': -1.8798633813858032,
 'longsleeve': -2.3595328330993652,
 'skirt': -2.6120944023132324,
 'shirt': -2.8261775970458984,
 'shoes': -3.6483089923858643,
 'hat': -4.756310939788818,
 't-shirt': -4.852035999298096}